# Setup

In [ ]:
!pip install nltk

In [ ]:
import numpy as np
import sklearn
import pandas as pd

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# constants and global variables
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CIL/Dataset/{}'

# if set to True the preprocessing for the bert model will be done, otherswise 
# the preprocessig for w2v and Tf-idf will be performed
is_bert_preprocessing_enabled = True

# Read Data

In [ ]:
tweets = []
labels = []

def load_tweets(filename, label):
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            tweets.append(line.rstrip())
            labels.append(label)

# load training tweets   
load_tweets(DATA_PATH.format('train_neg_full.txt'), 0)
load_tweets(DATA_PATH.format('train_pos_full.txt'), 1)
# Convert to NumPy array to facilitate indexing
print(f'{len(tweets)} training/dev tweets loaded')
tweets = np.array(tweets)
labels = np.array(labels)

# load the test file
f = open(DATA_PATH.format('test_data.txt'), 'r', encoding='utf-8')
X_test = []
for line in f:
  X_test.append(line.split(',')[1].rstrip())
X_test = np.array(X_test)
print(f'{len(X_test)} test tweets loaded')


for i in range(10):
    print(tweets[i])
    print(labels[i])

for i in range(10):
  print(X_test[i])

2500000 training/dev tweets loaded
10000 test tweets loaded
vinco tresorpack 6 ( difficulty 10 of 10 object : disassemble and reassemble the wooden pieces this beautiful wo ... <url>
0
glad i dot have taks tomorrow ! ! #thankful #startho
0
1-3 vs celtics in the regular season = were fucked if we play them in the playoffs
0
<user> i could actually kill that girl i'm so sorry ! ! !
0
<user> <user> <user> i find that very hard to believe im afraid
0
wish i could be out all night tonight ! <user>
0
<user> i got kicked out the wgm
0
rt <user> <user> <user> yes she is ! u tell it ! my lips are closed okay
0
why is she so perfect <url>
0
<user> hi harry ! did u havea good time in aus ? i didnt get 2 see u maybe next year ! follow me back if u can , would bea dreamcome truex
0
sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air
<user> shucks well i work all week so now i can't come cheer you on ! oh and put those batteries in your calculator ! ! !
i cant stay away fro

# Preprocess

In [ ]:
# preprocess for bert
if is_bert_preprocessing_enabled:
  pass
  # # Lowercase sentence
  # tweets = list(map(lambda tweet : tweet.lower(), tweets))
  # X_test = list(map(lambda tweet : tweet.lower(), X_test))

  # # remove hashtags
  # import re
  # tweets = list(map(lambda tweet : re.sub("#\w+", "", tweet), tweets))
  # X_test = list(map(lambda tweet : re.sub("#\w+", "", tweet), X_test))

  # remove duplicates from the training data (decreases accuracy)
  # print(len(tweets))
  # tweets_df = pd.DataFrame({'tweets':tweets, 'labels':labels}).drop_duplicates(subset=['tweets'], keep='last')
  # tweets = tweets_df["tweets"].to_numpy()
  # labels = tweets_df["labels"].to_numpy()
  # print(len(tweets))

  # split each tweet into separate words
  # tweets = list(map(lambda tweet : tweet.split(), tweets))
  # X_test = list(map(lambda tweet : tweet.split(), X_test))
  
  # remove user, url and other commond words (decreases accuracy)
  # forbidden_words = ["<url>", "<user>"]
  # tweets = list(map(lambda tweet : [w for w in tweet if not w in forbidden_words], tweets))
  # X_test = list(map(lambda tweet : [w for w in tweet if not w in forbidden_words], X_test))

  # from nltk.tokenize import RegexpTokenizer
  # # Tokenize sentence
  # tokenizer = RegexpTokenizer(r'\w+')
  # tweets = list(map(lambda tweet : tokenizer.tokenize(tweet), tweets))
  # X_test = list(map(lambda tweet : tokenizer.tokenize(tweet), X_test))

  # Remove stopwords (decrease accuracy)
  # from nltk.corpus import stopwords
  # stopwords_set = stopwords.words('english')
  # tweets = list(map(lambda tweet  : [w for w in tweet if not w in stopwords_set], tweets))
  # X_test = list(map(lambda tweet  : [w for w in tweet if not w in stopwords_set], X_test))

  # remove numbers (decreases accuracy)
  # import re
  # tweets = list(map(lambda tweet : [w for w in tweet if re.match("([0-9])+", w) == None], tweets))
  # X_test = list(map(lambda tweet : [w for w in tweet if re.match("([0-9])+", w) == None], X_test))

  # # join to back the tweets into a phrase
  # tweets = list(map(lambda tweet : " ".join(tweet), tweets))
  # X_test = list(map(lambda tweet : " ".join(tweet), X_test))



In [ ]:
# preprocess for non bert
if not is_bert_preprocessing_enabled:
  # Lowercase sentence
  tweets = list(map(lambda tweet : tweet.lower(), tweets))
  X_test = list(map(lambda tweet : tweet.lower(), X_test))

  from nltk.tokenize import RegexpTokenizer
  # Tokenize sentence
  tokenizer = RegexpTokenizer(r'\w+')
  tweets = list(map(lambda tweet : tokenizer.tokenize(tweet), tweets))
  X_test = list(map(lambda tweet : tokenizer.tokenize(tweet), X_test))

  # remove hashtags
  tweets = list(map(lambda tweet : [w for w in tweet if not w.startswith("#")], tweets))
  X_test = list(map(lambda tweet : [w for w in tweet if not w.startswith("#")], X_test))

  from nltk.corpus import stopwords
  # Remove stopwords
  stopwords_set = stopwords.words('english')
  tweets = list(map(lambda tweet  : [w for w in tweet if not w in stopwords_set], tweets))
  X_test = list(map(lambda tweet  : [w for w in tweet if not w in stopwords_set], X_test))

  from nltk.stem import WordNetLemmatizer
  # Lemmatize
  lemmatizer = WordNetLemmatizer()
  tweets = list(map(lambda tweet : [lemmatizer.lemmatize(w) for w in tweet], tweets))
  X_test = list(map(lambda tweet : [lemmatizer.lemmatize(w) for w in tweet], X_test))

  # remove numbers
  import re
  tweets = list(map(lambda tweet : [w for w in tweet if re.match("([0-9])+", w) == None], tweets))
  X_test = list(map(lambda tweet : [w for w in tweet if re.match("([0-9])+", w) == None], X_test))

  # remove user, url and other commond words
  forbidden_words = ["url", "user"]
  tweets = list(map(lambda tweet : [w for w in tweet if not w in forbidden_words], tweets))
  X_test = list(map(lambda tweet : [w for w in tweet if not w in forbidden_words], X_test))

  # # remove duplicates from the training data
  # tweets = list(map(lambda tweet : " ".join(tweet), tweets))
  # print(len(tweets))
  # tweets_df = pd.DataFrame({'tweets':tweets, 'labels':labels}).drop_duplicates(subset=['tweets'], keep='last')
  # tweets = tweets_df["tweets"].to_numpy()
  # labels = tweets_df["labels"].to_numpy()
  # print(len(tweets))

  # join back the tweets into a phrase
  X_test = list(map(lambda tweet : " ".join(tweet), X_test))
  tweets = list(map(lambda tweet : " ".join(tweet), tweets))


In [ ]:
for i in range(20):
  print(tweets[i])

print("\n\n\n")

for i in range(20):
  print(X_test[i])

vinco tresorpack 6 ( difficulty 10 of 10 object : disassemble and reassemble the wooden pieces this beautiful wo ... <url>
glad i dot have taks tomorrow ! ! #thankful #startho
1-3 vs celtics in the regular season = were fucked if we play them in the playoffs
<user> i could actually kill that girl i'm so sorry ! ! !
<user> <user> <user> i find that very hard to believe im afraid
wish i could be out all night tonight ! <user>
<user> i got kicked out the wgm
rt <user> <user> <user> yes she is ! u tell it ! my lips are closed okay
why is she so perfect <url>
<user> hi harry ! did u havea good time in aus ? i didnt get 2 see u maybe next year ! follow me back if u can , would bea dreamcome truex
introduction to programming with c + + ( 2nd edition this solid foundation in the basics of c + + programming will ... <url>
introduction to programming with c + + ( 2nd edition this solid foundation in the basics of c + + programming will ... <url>
introduction to programming with c + + ( 2nd editi

In [ ]:
# shuffle the training data
from sklearn.utils import shuffle

X_train, y_train = shuffle(tweets, labels, random_state=84)

In [ ]:
# save X data to a file
def save_X_data(file_name, X):
  f = open(DATA_PATH.format(file_name), "w")
  for element in X:
      # f.write(" ".join(element) + "\n")
      f.write(element + "\n")
  f.close()

save_X_data("X_train_processed_bert.txt" if is_bert_preprocessing_enabled else "X_train_processed.txt", X_train)
save_X_data("X_test_processed_bert.txt" if is_bert_preprocessing_enabled else "X_test_processed.txt", X_test)

In [ ]:
# save labels to file
f = open(DATA_PATH.format("y_train.txt"), "w")
for label in y_train:
  f.write(str(label) + "\n")
f.close()

# INITIAL EXPLORATION

In [ ]:
# get number of positive and negative tweets
print(f"In our dataset there are {(labels == 0).sum()} negative tweets")
print(f"In our dataset there are {(labels == 1).sum()} positive tweets")

In our dataset there are 1250000 negative tweets
In our dataset there are 1250000 positive tweets


In [ ]:
# get the average length of positive and negative examples
def get_average_length(target_label):
  len_tweets = []
  for tweet, label in zip(tweets, labels):
    if label == target_label:
      len_tweets.append(len(tweet))
  
  return np.array(len_tweets).mean()

# NOTE: postive tweets seem to be longer on average than negative tweets.
# TODO: check if this thing can be used at our advantange in training
print(f"The average length of tweets with a negative sentiment is: {get_average_length(0)}")
print(f"The average length of tweets with a positive sentiment is: {get_average_length(1)}")

The average length of tweets with a negative sentiment is: 84.0923984
The average length of tweets with a positive sentiment is: 67.835752


In [ ]:
# get most frequent words in positive and negative examples
def count_words(target_label):
  words_occurrences = {}
  for tweet, label in zip(tweets, labels):
    if label == target_label:
      for word in tweet:
        words_occurrences[word] = words_occurrences.get(word, 0) + 1

  return words_occurrences


from collections import Counter

tweets = list(map(lambda x : x.split, tweets))
# negative tweets
print("NEGATIVE TWEETS:")
negative = count_words(0)
print(dict(Counter(negative).most_common(100)))
# print uniques words in the negative tweets
print(len(negative))

# positive tweets
print("\n\n\n POSITIVE TWEETS:")
positive = count_words(1)
print(dict(Counter(positive).most_common(100)))
# print uniques words in the positive tweets
print(len(positive))

NEGATIVE TWEETS:
{' ': 20178690, 'e': 8558475, 't': 6041262, 'a': 5942525, 'o': 5902744, 'i': 5340092, 'r': 5141301, 's': 5120700, 'n': 4565219, 'l': 3757830, 'h': 3265654, 'u': 3196129, 'd': 2633209, 'm': 2422797, 'c': 2265983, 'y': 1825793, '.': 1786193, 'g': 1716708, 'w': 1666053, 'p': 1635392, 'f': 1424695, 'b': 1369047, 'k': 1041608, '<': 1041060, '>': 1022710, 'v': 766399, '(': 552651, "'": 468998, ',': 406069, '!': 368087, '1': 321188, '-': 316038, '0': 314103, 'x': 294188, '2': 280944, 'j': 196039, '3': 176580, ':': 157767, '5': 153606, '"': 150624, '#': 147696, '?': 140983, '4': 128185, '6': 119481, 'z': 115971, '/': 109841, '8': 96389, '7': 85965, ')': 84336, '9': 81105, 'q': 57020, '&': 44897, '*': 25457, '_': 14591, '|': 13706, ';': 9660, '+': 8010, '[': 6797, '$': 6661, ']': 6312, '=': 6065, '%': 5482, '~': 5124, '^': 3690, '@': 2238, '\\': 2049, '`': 1211, '{': 698, '}': 633, '“': 28, '®': 23, '’': 13, '×': 5, '°': 4, '♥': 4, '\x13': 2, '‘': 2, '”': 2, '\t': 2, '►': 2, '\